<a href="https://colab.research.google.com/github/dramfin/Simple-SMA-on-Colab/blob/main/Simple_SMA_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install backtrader alpaca-trade-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.8 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp311-cp311-linux_x86_64.whl size=15688 sha256=f988fd5faddd986b647394ae64ff3b699799232844855c8f331b8f87de3e1295
  Stored in directory: /root/.cache/pip/wheels/f6/35/da/ed9b26b510235e00e3a3c3bab7bad97b59214729662255ab3d
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.1.1
    Uninstalling msgpack-1.1.1:
      Successfully uninstalled msgpack-1.1.1
  Attempting uninstal

In [70]:
import os
from google.colab import userdata

api_key = userdata.get('APCA_API_KEY_ID')
api_secret = userdata.get('APCA_API_SECRET_KEY')

if not api_key or not api_secret:
    print("Alpaca API credentials not found in environment variables. Verify variable names, and upload.")
else:
    print("Alpaca API credentials successfully loaded from environment variables.")

Alpaca API credentials successfully loaded from environment variables.


In [71]:
import backtrader as bt

class SMACrossover(bt.Strategy):
    params = (
        ('pfast', 10),
        ('pslow', 70),
    )

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sellprice = None # Added sellprice

        self.sma_fast = bt.ind.SMA(self.datas[0], period=self.p.pfast)
        self.sma_slow = bt.ind.SMA(self.datas[0], period=self.p.pslow)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            #Buy Orders Below:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            #Sell Orders Below:
            elif order.issell():
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                self.sellprice = order.executed.price # Capture sell price
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        # Log trade details including buy/sell prices and profit/loss
        self.log(f'TRADE CLOSED - Buy Price: {self.buyprice:.2f}, Sell Price: {self.sellprice:.2f}, GROSS PROFIT: {trade.pnl:.2f}, NET PROFIT: {trade.pnlcomm:.2f}')


    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            if self.sma_fast[0] > self.sma_slow[0] and self.sma_fast[-1] <= self.sma_slow[-1]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.sma_fast[0] < self.sma_slow[0] and self.sma_fast[-1] >= self.sma_slow[-1]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

In [72]:
import backtrader as bt
import alpaca_trade_api as tradeapi
import pandas as pd

if api_key is None or api_secret is None:
    print("Alpaca API credentials not found. Cannot fetch data.")
else:
    try:
        api = tradeapi.REST(api_key, api_secret, base_url='https://paper-api.alpaca.markets')

        # Define parameters
        ticker = 'AAPL'
        timeframe = '1D' # Daily Time Frame
        start_date = '2022-01-01'
        end_date = '2023-01-01'

        # Fetch historical data
        # Time Frame is one year, daily prices.
        bars = api.get_bars(ticker, timeframe, start=start_date, end=end_date).df

        # Ensure index is datetime
        bars.index = pd.to_datetime(bars.index)

        # Create backtrader data feed
        data = bt.feeds.PandasData(dataname=bars)
        print("Backtrader data feed created successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

Backtrader data feed created successfully.


In [73]:
# Instantiate Cerebro engine
cerebro = bt.Cerebro()

# Add data feed
if 'data' in locals() and data is not None:
    cerebro.adddata(data)
    print("Data feed added to Cerebro.")
else:
    print("Data feed not available. Cannot run backtest.")

# Add strategy
cerebro.addstrategy(SMACrossover)
print("SMACrossover strategy added to Cerebro.")

# Set starting cash
cerebro.broker.setcash(10000.0)
print("Starting cash set to 10000.0")

# Run backtest
if 'data' in locals() and data is not None:
    print("Running backtest...")
    results = cerebro.run()
    print("Backtest finished.")
else:
    print("Backtest cannot run without data feed.")

Data feed added to Cerebro.
SMACrossover strategy added to Cerebro.
Starting cash set to 10000.0
Running backtest...
2022-04-12, Close, 167.66
2022-04-13, Close, 170.40
2022-04-14, Close, 165.29
2022-04-18, Close, 165.07
2022-04-19, Close, 167.40
2022-04-20, Close, 167.23
2022-04-21, Close, 166.42
2022-04-22, Close, 161.79
2022-04-25, Close, 162.88
2022-04-26, Close, 156.80
2022-04-27, Close, 156.57
2022-04-28, Close, 163.64
2022-04-29, Close, 157.65
2022-05-02, Close, 157.96
2022-05-03, Close, 159.48
2022-05-04, Close, 166.02
2022-05-05, Close, 156.77
2022-05-06, Close, 157.28
2022-05-09, Close, 152.06
2022-05-10, Close, 154.51
2022-05-11, Close, 146.50
2022-05-12, Close, 142.56
2022-05-13, Close, 147.11
2022-05-16, Close, 145.54
2022-05-17, Close, 149.24
2022-05-18, Close, 140.82
2022-05-19, Close, 137.35
2022-05-20, Close, 137.59
2022-05-23, Close, 143.11
2022-05-24, Close, 140.36
2022-05-25, Close, 140.52
2022-05-26, Close, 143.78
2022-05-27, Close, 149.64
2022-05-31, Close, 148.84

In [91]:
from logging import log
if 'results' in locals() and results:
    # Access the strategy instance (assuming only one strategy)
    strategy = results[0]

    # Print key performance metrics
    print("\n--- Backtest Results ---")
    print(f"Initial Portfolio Value: {cerebro.broker.startingcash:.2f}") # Print initial value
    print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")

else:
    print("Backtest results are not available. The backtest may not have run successfully.")


--- Backtest Results ---
Initial Portfolio Value: 10000.00
Final Portfolio Value: 9994.27


This notebook demonstrates a simple trading strategy using Backtrader and fetching historical data from Alpaca.

The notebook is structured as follows:

1.  **Installation**: Installs the necessary libraries.
2.  **Alpaca API Setup**: Loads Alpaca API credentials from Colab secrets.
3.  **Strategy Definition**: Defines the trading strategy (`SMACrossover`).
4.  **Data Fetching**: Fetches historical data using the Alpaca API.
5.  **Backtrader Setup and Run**: Sets up and runs the backtest with the defined strategy and data.
6.  **Results Analysis**: Prints the backtest results.

### 1. Installation

This cell installs the required libraries: `backtrader` for backtesting and `alpaca-trade-api` for fetching historical data.

### 2. Alpaca API Setup

This cell loads the Alpaca API key and secret from Colab secrets. It's important to store your API credentials securely and not directly in the code.

### 3. Strategy Definition

This cell defines the `SMACrossover` trading strategy.

-   **`__init__`**: Initializes the strategy, including the fast and slow Simple Moving Averages (SMAs) and variables to track trades.
-   **`notify_order`**: This method is called when an order's status changes (submitted, accepted, completed, canceled, etc.). It logs the details of executed buy and sell orders.
-   **`notify_trade`**: This method is called when a trade is closed. It logs the details of the closed trade, including the buy and sell prices and the profit/loss.
-   **`next`**: This method is called for each bar in the data feed. It implements the trading logic:
    -   If there is no open position and the fast SMA crosses above the slow SMA, a buy order is created.
    -   If there is an open position and the fast SMA crosses below the slow SMA, a sell order is created.
-   **`log`**: A helper function to print log messages with the date.

### 4. Data Fetching

This cell fetches historical daily data for AAPL from January 1, 2022, to January 1, 2023, using the Alpaca API and creates a Backtrader data feed from it.

### 5. Backtrader Setup and Run

This cell sets up the Backtrader engine (`cerebro`), adds the data feed and the `SMACrossover` strategy, sets the initial cash, and runs the backtest.

### 6. Results Analysis

This cell prints the final portfolio value after the backtest.